# LLM - RAG (대규모 언어 모델 - 검색 증강 생성)

# 개요
이 데모는 GPT-4 모델을 검색 증강 생성(RAG) 설정에서 사용하는 방법을 보여줍니다. \
모델은 자연어 질문을 분석하고, 지식 기반에서 관련 정보를 검색한 후, 검색된 정보를 기반으로 답변을 제공합니다.



## 검색 증강 생성(RAG)

LLM 기반 챗봇은 사용자 프롬프트를 처리하여 응답을 생성합니다.  
이는 상호작용을 위해 설계되었으며 다양한 주제에 대해 사용자와 소통합니다.  
그러나 응답은 제공된 컨텍스트와 기본 교육 데이터에 제한됩니다.  
예를 들어, GPT-4의 지식 컷오프는 2021년 9월로, 이 시점 이후에 발생한 이벤트에 대한 지식이 부족합니다.  
또한, LLM을 훈련하는 데 사용된 데이터는 개인 노트나 회사의 제품 매뉴얼과 같은 기밀 정보를 제외합니다.

### RAG(Retrieval-Augmented Generation)를 사용하는 이유는 무엇인가요?

- **정보 풍부성:** 텍스트 응답이 최신 상태를 유지하고 최신 정보를 제공할 수 있습니다. 따라서 내부 지식 베이스에 액세스하여 도메인별 작업 성능을 향상시킵니다.

- **허위 정보 감소:** 지식 베이스의 **검증 가능한 데이터**를 활용하여 사용자 쿼리에 대한 컨텍스트를 제공합니다.

- **비용 효율성:** LLM을 세부 조정하는 것보다 경제적입니다.

### 🔍 RAG(Retrieval Augmented Generation) 작동 방식

**1. 문서 수집**  
- PDF, 매뉴얼, 사내 문서 등 지식 기반이 되는 원본 데이터를 준비한다.

**2. 문서 임베딩 생성**  
- 문서를 Chunk 단위로 나누고  
- Embedding Model을 사용해 의미 기반 벡터로 변환한다.  
- 생성된 벡터는 Vector DB에 저장된다.

**3. 사용자 질의 입력**  
- 사용자가 질문을 입력한다.  
  예: “XYZ 데이터를 기반으로 퀴즈 10개 만들어줘.”

**4. 질의 임베딩 생성**  
- 동일한 Embedding Model로 Query를 벡터로 변환한다.

**5. 벡터 검색**  
- Query 벡터와 의미적으로 유사한 문서 조각을 Vector DB에서 검색한다.

**6. 컨텍스트 반환**  
- Top-k 검색 결과(문서 조각)를 LLM에 전달할 컨텍스트로 준비한다.

**7. LLM 입력 구성**  
- 사용자 질문 + 검색된 문서 조각(Context)을 합쳐 Prompt를 구성한다.

**8. LLM이 답변 생성**  
- 검색된 컨텍스트를 반영해 사실 기반(Factual) 응답을 생성한다.

**9. 사용자에게 결과 반환**  
- 최종 응답을 사용자에게 전달한다.

---

![Deploy](image/how-rag-works.png)





In [2]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-12-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

### 자연어 질문으로 시작하기

In [3]:
input = "2025년 대한민국 또는 글로벌 IT 트렌드(GenAI, Security, Cloud)의 최신 기술 보고서를 요약해줘."

### 1단계: GPT-4o: 이 질문에 답하기 위해 무엇이 필요한가요?

In [4]:
system_prompt=f'''다음 질문에 가능한 한 최선을 다해 답하세요. 다음 도구에 액세스할 수 있습니다:

웹 검색: 웹을 사용하여 정보를 찾습니다.

아래 형식을 엄격하게 따라야 합니다:

질문: 반드시 답해야 하는 입력 질문
생각: 무엇을 해야 할지 항상 생각해야 합니다.
행동: 취해야 할 행동, [조사] 중 하나이어야 합니다.
행동 입력: 행동에 대한 입력

시작!
'''

user_prompt = f'''
Question: {input}
Thought:'''

In [5]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

Thought: 2025년 IT 트렌드(GenAI, Security, Cloud)와 관련된 최신 기술 보고서를 확인하여 요약해야 한다. 이를 위해 국내외 주요 IT 보고서를 조사할 필요가 있다.  
Action: 조사  
Action Input: 2025년 대한민국 또는 글로벌 IT 트렌드(Generative AI, Security, Cloud) 관련 최신 기술 보고서 요약


### 2단계: 더 많은 세부 정보를 위해 웹 검색


계속하기 전에 구독에서 검색 서비스를 생성하세요 [검색 리소스 생성](https://serpapi.com). \
이 샘플은 쿼리를 사용하여 웹 검색 API에 호출을 수행하고 관련 웹 검색 결과를 반환합니다.

In [6]:

import serpapi
import json

params = {
  "engine": "google",
  "q": "2025년 대한민국 또는 글로벌 IT 트렌드(GenAI, Security, Cloud)의 최신 기술 보고서를 요약해줘.",
  "api_key": os.getenv("SERP_API_KEY"),
}
print()

response = serpapi.search(params)

# response 객체를 JSON 문자열로 변환 후, ensure_ascii=False로 한글이 깨지지 않게 출력
print(json.dumps(response.data, ensure_ascii=False, indent=2))

context = ''
organic_results = response["organic_results"]

for i, result in enumerate(organic_results):
    title = result.get("title")
    link = result.get("link")
    snippet = result.get("snippet")
    # Ensure the title is printed as a proper Korean string
    if isinstance(snippet, bytes):
        snippet = snippet.decode('utf-8')
    if isinstance(title, bytes):
        title = title.decode('utf-8')
    print(f"Result {i+1}:")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print(f"Snippet:{snippet}")
    context += snippet + '\n'


{
  "search_metadata": {
    "id": "69197f3e410cdaddb6be6def",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/68d338594603485c/69197f3e410cdaddb6be6def.json",
    "pixel_position_endpoint": "https://serpapi.com/searches/68d338594603485c/69197f3e410cdaddb6be6def.json_with_pixel_position",
    "created_at": "2025-11-16 07:37:34 UTC",
    "processed_at": "2025-11-16 07:37:34 UTC",
    "google_url": "https://www.google.com/search?q=2025%EB%85%84+%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD+%EB%98%90%EB%8A%94+%EA%B8%80%EB%A1%9C%EB%B2%8C+IT+%ED%8A%B8%EB%A0%8C%EB%93%9C%28GenAI%2C+Security%2C+Cloud%29%EC%9D%98+%EC%B5%9C%EC%8B%A0+%EA%B8%B0%EC%88%A0+%EB%B3%B4%EA%B3%A0%EC%84%9C%EB%A5%BC+%EC%9A%94%EC%95%BD%ED%95%B4%EC%A4%98.&oq=2025%EB%85%84+%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD+%EB%98%90%EB%8A%94+%EA%B8%80%EB%A1%9C%EB%B2%8C+IT+%ED%8A%B8%EB%A0%8C%EB%93%9C%28GenAI%2C+Security%2C+Cloud%29%EC%9D%98+%EC%B5%9C%EC%8B%A0+%EA%B8%B0%EC%88%A0+%EB%B3%B4%EA%B3%A0%EC%84%9C%EB%A5%BC+%EC%9A%

### 추가 처리를 위한 수집된 정보

In [7]:
print(context)

CIO를 비롯한 기업 리더들이 앞으로는 분산된 클라우드 환경을 고려한 IT 전략을 설계해야 합니다. 그 전략에는 운영 복잡성 관리, 멀티 플랫폼 보안 및 ...
이 글에서는 가트너가 선정한 2025년 10대 전략 기술 트렌드를 중심으로 AI, 컴퓨팅, 보안 영역의 주요 변화와 산업 전망을 종합적으로 분석합니다.
요약 2025년 8월 현재 IT 기술 생태계는 AI의 급속한 발전과 성숙, 클라우드 네이티브 아키텍처의 표준화, 그리고 보안 중심의 개발 문화로 특징지어 ...
AI 기반 방어 체계, 제로 트러스트 모델, 강화된 위협 탐지 능력 등 클라우드 보안 환경을 주도하는 주요 트렌드를 확인하세요. CS-101_Cloud.svg.
본 글에서는 Google Cloud가 발표한 1,001건의 실제 AI 활용 사례를 심층 분석하여, 산업별·유형별 트렌드를 파악하고 데이터 사이언스 및 머신러닝 관점 ...
보고서는 향후 불확실성 증가로 인한 IT 지출 감소가 예상됨에도 불구하고 AI가 기업 내 주요 화두가 되며 클라우드 관련 투자가 지속될 것이라고 말했다.
2025년 최근 IT 업계 트렌드 10 가지 · 에이전트형 AI(Agentic AI) · AI 거버넌스 플랫폼 · 가짜 정보 보안 · 포스트 양자 암호 · 환경에 녹는 인텔리전스.
2025년 글로벌 IT 트렌드: 하이브리드 클라우드, 지속 가능한 IT, AX시대로의 전환, AIaaS, 제로 트러스트 보안까지! 올해 떠오르는 IT 트렌드 핵심 ...
2025년 상반기 주요 빅테크 기업들의 키노트를 중심으로, 최신 기술의 트렌드와 전략적 방향을 살펴보고자 합니다.
이 글에서는 2025년에 꼭 알아야 할 5가지 핵심 IT 이슈를 소개하고, 국내외 실제 적용 사례까지 함께 정리해드릴게요. 관련 자료 바로 보기 2025년 ...



In [8]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### 3단계: GPT-4o: 이 질문에 답하기 위해 무엇을 해야 하나요? (다시)

In [10]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=1000,)

print(response.choices[0].message.content)

네, 2025년 대한민국 및 글로벌 IT 트렌드(GenAI, Security, Cloud)에 대한 최신 기술 보고서를 핵심적으로 요약해드리겠습니다.

---

### 1. 생성형 AI(Generative AI, GenAI)

- **급속한 발전 및 성숙**: GenAI가 기업 내 주요 업무와 서비스 혁신의 중심으로 자리잡음. 자동화, 콘텐츠 생성, 데이터 분석 등 산업 전반에서 광범위하게 적용. 구글, MS, 오픈AI 등 주요 빅테크 기업의 AI 기능 고도화 및 통합 가속화.
- **에이전트형 AI**: 복잡한 업무를 스스로 수행하는 ‘에이전트형 AI’가 실질적 생산성 도약을 주도. 기업 업무 자동화, 창의적 협업 등에서 실제 사례 ↑.
- **AI 거버넌스**: 생성AI의 윤리, 투명성, 품질 관리에 대한 플랫폼·솔루션 투자가 확대. AI 규제 및 정책 환경도 빠르게 변화 중.
- **실제 활용 확산**: Google Cloud의 1,001건 사례 분석 등 각 산업별 맞춤형 도입 활성화. 데이터 사이언스와 머신러닝을 바탕으로 다양한 비즈니스 모델 창출.

---

### 2. 클라우드(Cloud & Hybrid)

- **분산 및 하이브리드 클라우드**: 여러 클라우드 플랫폼(퍼블릭·프라이빗·엣지)을 혼합하는 전략이 표준화. 미래 지향적 IT 아키텍처는 하이브리드/멀티 클라우드 채택이 필수.
- **클라우드 네이티브**: 인프라 및 애플리케이션의 클라우드 친화적(Cloud-Native) 설계가 일반화. NoOps/자동화, DevSecOps, 플랫폼 엔지니어링이 트렌드.
- **지속 가능한 IT**: 친환경 IT, 탄소 감축 목표 달성을 위한 클라우드 전환과 운영 효율화가 강조됨. 리더들은 환경 요소(E)의 경영전략에 IT를 적극 통합 중.

---

### 3. 보안(Security)

- **제로 트러스트 모델**: 모든 접속과 데이터에 대해 '기본 불신'(Zero-Trust) 원칙 적용. 사용자·디바이스·애플리케이션 단계별 인증과 권한 관리 강화.
- *

### GPT-4o: 이제 질문에 대한 최종 답변을 얻었습니다.

#### 탐색할 사용 사례
1. **지식 관리** \
내부 문서, 매뉴얼 및 정책에서 정보를 검색하고 제공하여 기업 지식에 대한 접근성을 향상시킵니다.

2. **비즈니스 인텔리전스 및 분석** \
시장 분석가가 다양한 소스에서 관련 시장 데이터, 보고서 및 통찰력을 검색할 수 있도록 지원합니다.

3. **법률 및 준수** \
법률 전문가가 쿼리에 따라 관련 판례, 법령 및 법적 선례를 검색할 수 있도록 지원합니다.